## Data Wrangling

In [1575]:
import pandas as pd
import os
import re
import numpy as np

In [ ]:
path="../knoweldge_system/artifacts/benimellal"

In [ ]:
DATASET_DIR = "./datasets/benimellal"

# ---- 1) Helper: parse city name from filename like "sale_24.csv" or "casa_2024.csv"
def parse_city_from_filename(filename: str) -> str:
    base = os.path.splitext(filename)[0]          # sale_24
    parts = base.split("_")
    city = parts[0].strip().lower()
    return city

# ---- 2) Read all CSVs, add city_id, then combine
dfs = []
city_to_id = {}   # consistent mapping city -> integer id

for filename in os.listdir(DATASET_DIR):
    if not filename.lower().endswith(".csv"):
        continue

    city = parse_city_from_filename(filename)

    # assign a stable id per city (incremental)
    if city not in city_to_id:
        city_to_id[city] = len(city_to_id)

    df = pd.read_csv(os.path.join(DATASET_DIR, filename))

    # Add city_id (and optionally city name)
    df["city"] = city
    df["city_id"] = city_to_id[city]

    dfs.append(df)

# combine
global_df = pd.concat(dfs, ignore_index=True)

# ---- 3) Ensure date column is datetime (change "date" if your column name differs)
# If your date column is named differently (e.g. "timestamp"), replace it below.
global_df["DATE"] = pd.to_datetime(global_df["DATE"], errors="coerce")

# ---- 4) Sort by city_id then date
global_df = global_df.sort_values(["city_id", "DATE"]).reset_index(drop=True)

print("Combined shape:", global_df.shape)
print("City mapping:", city_to_id)
print(global_df.head(10))


Combined shape: (3112, 30)
City mapping: {'benimellal': 0}
       STATION       DATE   LATITUDE  LONGITUDE  ELEVATION             NAME  \
0  60191099999 2017-01-01  32.366667       -6.4      472.0  BENI MELLAL, MO   
1  60191099999 2017-01-02  32.366667       -6.4      472.0  BENI MELLAL, MO   
2  60191099999 2017-01-03  32.366667       -6.4      472.0  BENI MELLAL, MO   
3  60191099999 2017-01-04  32.366667       -6.4      472.0  BENI MELLAL, MO   
4  60191099999 2017-01-05  32.366667       -6.4      472.0  BENI MELLAL, MO   
5  60191099999 2017-01-06  32.366667       -6.4      472.0  BENI MELLAL, MO   
6  60191099999 2017-01-07  32.366667       -6.4      472.0  BENI MELLAL, MO   
7  60191099999 2017-01-08  32.366667       -6.4      472.0  BENI MELLAL, MO   
8  60191099999 2017-01-09  32.366667       -6.4      472.0  BENI MELLAL, MO   
9  60191099999 2017-01-10  32.366667       -6.4      472.0  BENI MELLAL, MO   

   TEMP  TEMP_ATTRIBUTES  DEWP  DEWP_ATTRIBUTES  ...   MAX  MAX_ATTRIBU

In [1578]:
global_df.drop(columns=["city_id","city"], inplace=True)

In [1579]:
global_df.columns

Index(['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'TEMP',
       'TEMP_ATTRIBUTES', 'DEWP', 'DEWP_ATTRIBUTES', 'SLP', 'SLP_ATTRIBUTES',
       'STP', 'STP_ATTRIBUTES', 'VISIB', 'VISIB_ATTRIBUTES', 'WDSP',
       'WDSP_ATTRIBUTES', 'MXSPD', 'GUST', 'MAX', 'MAX_ATTRIBUTES', 'MIN',
       'MIN_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES', 'SNDP', 'FRSHTT'],
      dtype='object')

In [1580]:
# Columns we want to keep
FINAL_COLUMNS = [
    "DATE",
    "TEMP",
    "MAX",
    "MIN",
    "DEWP",
    "PRCP",
    "WDSP",
    "GUST",
    "VISIB",
    "city_id",
    "city",
]

# Drop all other columns
global_df = global_df.loc[:, [c for c in FINAL_COLUMNS if c in global_df.columns]]

In [1581]:
global_df.head()

,DATE,TEMP,MAX,MIN,DEWP,PRCP,WDSP,GUST,VISIB
0,2017-01-01,53.6,64.4,41.0,33.9,0.0,1.8,999.9,999.9
1,2017-01-02,54.0,68.0,33.8,34.8,0.0,2.4,999.9,999.9
2,2017-01-03,56.2,68.0,35.6,31.5,0.0,2.8,999.9,999.9
3,2017-01-04,56.8,69.8,37.4,32.9,0.0,3.4,999.9,999.9
4,2017-01-05,57.5,69.8,37.4,36.9,0.0,1.6,999.9,999.9


In [1582]:
global_df["DATE"] = pd.to_datetime(global_df["DATE"])
global_df = global_df.sort_values("DATE").set_index("DATE")

In [1583]:
full_idx = pd.date_range(
    start="2017-01-01",
    end="2025-08-24",
    freq="D"
)

global_df = global_df.reindex(full_idx)

In [1584]:
num_cols = [
    "TEMP","MAX","MIN","DEWP",
    "PRCP","WDSP","GUST","VISIB"
]

global_df[num_cols] = (
    global_df[num_cols]
    .interpolate(method="time")
    .ffill()
    .bfill()
)

In [1585]:
global_df.isna().sum()

TEMP     0
MAX      0
MIN      0
DEWP     0
PRCP     0
WDSP     0
GUST     0
VISIB    0
dtype: int64

In [1586]:
global_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3158 entries, 2017-01-01 to 2025-08-24
Freq: D
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   TEMP    3158 non-null   float64
 1   MAX     3158 non-null   float64
 2   MIN     3158 non-null   float64
 3   DEWP    3158 non-null   float64
 4   PRCP    3158 non-null   float64
 5   WDSP    3158 non-null   float64
 6   GUST    3158 non-null   float64
 7   VISIB   3158 non-null   float64
dtypes: float64(8)
memory usage: 222.0 KB


In [1587]:
missing_codes = {
    "TEMP": 9999.9,
    "MAX": 9999.9,
    "MIN": 9999.9,
    "DEWP": 9999.9,
    "VISIB": 999.9,
    "WDSP": 999.9,
    "GUST": 999.9,
    "PRCP": 99.99,
}

# Replace missing with NaN
for col, miss_val in missing_codes.items():
    if col in global_df.columns:
        global_df[col] = global_df[col].replace(miss_val, np.nan)

In [1588]:
global_df.isna().sum()

TEMP        0
MAX         0
MIN        13
DEWP        0
PRCP      137
WDSP        2
GUST     3067
VISIB     142
dtype: int64

In [1589]:
global_df.drop(columns=["GUST"], inplace=True)
global_df["PRCP"] = global_df["PRCP"].fillna(0.0)

In [1590]:
global_df.head()

,TEMP,MAX,MIN,DEWP,PRCP,WDSP,VISIB
2017-01-01,53.6,64.4,41.0,33.9,0.0,1.8,NaN
2017-01-02,54.0,68.0,33.8,34.8,0.0,2.4,NaN
2017-01-03,56.2,68.0,35.6,31.5,0.0,2.8,NaN
2017-01-04,56.8,69.8,37.4,32.9,0.0,3.4,NaN
2017-01-05,57.5,69.8,37.4,36.9,0.0,1.6,NaN


In [1591]:
global_df.rename(columns={
    "TEMP":"mean_temperature",
    "DEWP":"mean_dewPoint",
    "VISIB":"mean_visibility",
    "WDSP":"mean_windSpeed",
    "MAX":"max_temperature",
    "MIN":"min_temperature",
    "PRCP":"total_precipitation"
},inplace=True)

In [1592]:
global_df.head()

,mean_temperature,max_temperature,min_temperature,mean_dewPoint,total_precipitation,mean_windSpeed,mean_visibility
2017-01-01,53.6,64.4,41.0,33.9,0.0,1.8,NaN
2017-01-02,54.0,68.0,33.8,34.8,0.0,2.4,NaN
2017-01-03,56.2,68.0,35.6,31.5,0.0,2.8,NaN
2017-01-04,56.8,69.8,37.4,32.9,0.0,3.4,NaN
2017-01-05,57.5,69.8,37.4,36.9,0.0,1.6,NaN


In [1593]:
# --- Temperature columns (F → C)
temp_cols = [
    "mean_temperature",
    "max_temperature",
    "min_temperature",
    "mean_dewPoint",
]

for col in temp_cols:
    global_df[col] = (global_df[col] - 32) * 5 / 9


# --- Precipitation (inches → mm)
global_df["total_precipitation"] = global_df["total_precipitation"] * 25.4


# --- Wind speed (knots → m/s)
global_df["mean_windSpeed"] = global_df["mean_windSpeed"] * 0.514444


# --- Visibility (miles → km)
global_df["mean_visibility"] = global_df["mean_visibility"] * 1.60934

In [1594]:
global_df.describe()

,mean_temperature,max_temperature,min_temperature,mean_dewPoint,total_precipitation,mean_windSpeed,mean_visibility
count,3158.000000,3158.000000,3145.000000,3158.000000,3158.000000,3156.000000,3016.000000
mean,22.134104,28.354681,13.791839,7.570966,0.032172,2.350378,16.636712
std,7.520134,8.161114,49.765598,4.738460,0.562866,0.803184,84.685070
min,4.944444,7.000000,-2.777778,-11.555556,0.000000,0.411555,1.287472
25%,15.680556,22.000000,7.222222,4.347222,0.000000,1.749110,9.977908
50%,21.694444,28.000000,13.000000,8.166667,0.000000,2.212109,9.977908
75%,27.930556,34.361111,18.277778,11.222222,0.000000,2.777998,9.977908
max,41.166667,47.722222,2778.250000,18.166667,14.986000,8.642659,1549.949393


In [1595]:
global_df.to_csv(f"{path}/weather.csv", index=True)

## Feature Engineering

In [1412]:
# Extract calendar components
global_df["day_of_week"] = global_df.index.dayofweek   # 0=Mon, 6=Sun
global_df["day_of_year"] = global_df.index.dayofyear

# --- Weekly seasonality (optional but useful)
global_df["dow_sin"] = np.sin(2 * np.pi * global_df["day_of_week"] / 7)
global_df["dow_cos"] = np.cos(2 * np.pi * global_df["day_of_week"] / 7)

# --- Yearly seasonality (VERY IMPORTANT)
global_df["doy_sin"] = np.sin(2 * np.pi * global_df["day_of_year"] / 365.25)
global_df["doy_cos"] = np.cos(2 * np.pi * global_df["day_of_year"] / 365.25)

# Drop raw integer columns (not needed anymore)
global_df.drop(columns=["day_of_week", "day_of_year"], inplace=True)

In [1413]:
global_df.columns

Index(['mean_temperature', 'max_temperature', 'min_temperature',
       'mean_dewPoint', 'total_precipitation', 'mean_windSpeed',
       'mean_visibility', 'dow_sin', 'dow_cos', 'doy_sin', 'doy_cos'],
      dtype='object')

In [1414]:
LAG_COLS = [
    "mean_temperature",
    "max_temperature",
    "min_temperature",
    "mean_dewPoint",
    "total_precipitation",
    "mean_windSpeed",
]

LAGS = [1, 3, 7]

for col in LAG_COLS:
    for lag in LAGS:
        global_df[f"{col}_lag_{lag}"] = global_df[col].shift(lag)


In [1415]:
ROLL_WINDOWS = [3, 7]

# --- Temperature trends
for w in ROLL_WINDOWS:
    global_df[f"mean_temperature_roll_mean_{w}"] = (
        global_df["mean_temperature"]
        .rolling(window=w, min_periods=1)
        .mean()
    )

    global_df[f"max_temperature_roll_max_{w}"] = (
        global_df["max_temperature"]
        .rolling(window=w, min_periods=1)
        .max()
    )

# --- Precipitation accumulation
for w in ROLL_WINDOWS:
    global_df[f"total_precipitation_roll_sum_{w}"] = (
        global_df["total_precipitation"]
        .rolling(window=w, min_periods=1)
        .sum()
    )

# --- Wind trend
for w in ROLL_WINDOWS:
    global_df[f"mean_windSpeed_roll_mean_{w}"] = (
        global_df["mean_windSpeed"]
        .rolling(window=w, min_periods=1)
        .mean()
    )

In [1416]:
global_df.columns

Index(['mean_temperature', 'max_temperature', 'min_temperature',
       'mean_dewPoint', 'total_precipitation', 'mean_windSpeed',
       'mean_visibility', 'dow_sin', 'dow_cos', 'doy_sin', 'doy_cos',
       'mean_temperature_lag_1', 'mean_temperature_lag_3',
       'mean_temperature_lag_7', 'max_temperature_lag_1',
       'max_temperature_lag_3', 'max_temperature_lag_7',
       'min_temperature_lag_1', 'min_temperature_lag_3',
       'min_temperature_lag_7', 'mean_dewPoint_lag_1', 'mean_dewPoint_lag_3',
       'mean_dewPoint_lag_7', 'total_precipitation_lag_1',
       'total_precipitation_lag_3', 'total_precipitation_lag_7',
       'mean_windSpeed_lag_1', 'mean_windSpeed_lag_3', 'mean_windSpeed_lag_7',
       'mean_temperature_roll_mean_3', 'max_temperature_roll_max_3',
       'mean_temperature_roll_mean_7', 'max_temperature_roll_max_7',
       'total_precipitation_roll_sum_3', 'total_precipitation_roll_sum_7',
       'mean_windSpeed_roll_mean_3', 'mean_windSpeed_roll_mean_7'],
      d

In [1417]:
# --- Temperature changes (°C)
global_df["delta_temp_1d"] = (
    global_df["mean_temperature"].diff(1)
)

global_df["delta_temp_3d"] = (
    global_df["mean_temperature"].diff(3)
)


# --- Wind speed changes (m/s)
global_df["wind_increase_1d"] = (
    global_df["mean_windSpeed"].diff(1)
)

global_df["wind_increase_3d"] = (
    global_df["mean_windSpeed"].diff(3)
)


# --- Precipitation changes (mm)
global_df["precip_increase_1d"] = (
    global_df["total_precipitation"].diff(1)
)

## LSTM forecasting model
This section prepares the daily time series and trains a simple LSTM to forecast mean temperature.


In [1418]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

In [1419]:
LOOKBACK = 14
HORIZON = 7

TARGET_COLS = [
    "mean_temperature",
    "max_temperature",
    "min_temperature",
    "total_precipitation",
    "mean_windSpeed",
    "mean_dewPoint",
    "mean_visibility",
]


FEATURE_COLS = [
    # --- core current values
    "mean_temperature",
    "max_temperature",
    "min_temperature",
    "mean_dewPoint",
    "total_precipitation",
    "mean_windSpeed",
    "mean_visibility",

    # --- lag features
    "mean_temperature_lag_1",
    "mean_temperature_lag_3",
    "mean_temperature_lag_7",

    # --- rolling statistics
    "mean_temperature_roll_mean_3",
    "mean_temperature_roll_mean_7",
    "total_precipitation_roll_sum_3",
    "total_precipitation_roll_sum_7",

    # --- change features
    "delta_temp_1d",
    "delta_temp_3d",
    "wind_increase_1d",
    "precip_increase_1d",

    # --- seasonality
    "dow_sin",
    "dow_cos",
    "doy_sin",
    "doy_cos",
]

import numpy as np

def build_sequences_multi_target(df, feature_cols, target_cols, lookback=14, horizon=7):
    X, Y = [], []

    # Ensure chronological order
    df = df.sort_index()

    feature_values = df[feature_cols].values
    target_values = df[target_cols].values

    n_targets = len(target_cols)

    for i in range(lookback, len(df) - horizon):
        # Input window
        X.append(
            feature_values[i - lookback : i]
        )

        # Output: next HORIZON days for ALL targets
        future = target_values[i : i + horizon]          # (HORIZON, n_targets)
        Y.append(
            future.reshape(horizon * n_targets)
        )

    return np.array(X), np.array(Y)

In [1420]:
X, Y = build_sequences_multi_target(
    global_df,
    FEATURE_COLS,
    TARGET_COLS,
    lookback=LOOKBACK,
    horizon=HORIZON
)

print("X shape:", X.shape)
print("Y shape:", Y.shape)

X shape: (3137, 14, 22)
Y shape: (3137, 49)


In [1421]:
SPLIT_DATE = "2025-01-01"

train_df = global_df.loc[global_df.index < SPLIT_DATE].copy()
test_df  = global_df.loc[global_df.index >= SPLIT_DATE].copy()

print(
    train_df.index.min(), "→", train_df.index.max(),
    "| rows:", len(train_df)
)
print(
    test_df.index.min(), "→", test_df.index.max(),
    "| rows:", len(test_df)
)

2017-01-01 00:00:00 → 2024-12-31 00:00:00 | rows: 2922
2025-01-01 00:00:00 → 2025-08-24 00:00:00 | rows: 236


In [1422]:
# After train / test split
train_df = train_df.dropna().copy()
test_df  = test_df.dropna().copy()

In [1423]:
X_train, Y_train = build_sequences_multi_target(
    train_df,
    feature_cols=FEATURE_COLS,
    target_cols=TARGET_COLS,   # correct targets
    lookback=LOOKBACK,
    horizon=HORIZON
)

X_test, Y_test = build_sequences_multi_target(
    test_df,
    feature_cols=FEATURE_COLS,
    target_cols=TARGET_COLS,   # SAME ORDER (very important)
    lookback=LOOKBACK,
    horizon=HORIZON
)

In [1424]:
print("X_train:", X_train.shape)
print("Y_train:", Y_train.shape)

print("X_test :", X_test.shape)
print("Y_test :", Y_test.shape)

X_train: (2886, 14, 22)
Y_train: (2886, 49)
X_test : (215, 14, 22)
Y_test : (215, 49)


In [1425]:
np.isnan(X_train).any(), np.isnan(X_test).any()

(False, False)

In [1426]:
# Save shapes
n_train, t_steps, n_features = X_train.shape
n_test = X_test.shape[0]

# Flatten time dimension
X_train_2d = X_train.reshape(-1, n_features)
X_test_2d  = X_test.reshape(-1, n_features)

# Fit scaler on TRAIN only
feature_scaler = StandardScaler()
feature_scaler.fit(X_train_2d)

X_train_scaled = feature_scaler.transform(X_train_2d).reshape(
    n_train, t_steps, n_features
)

X_test_scaled = feature_scaler.transform(X_test_2d).reshape(
    n_test, t_steps, n_features
)

print("Train mean (≈0):", X_train_scaled.mean())
print("Train std  (≈1):", X_train_scaled.std())

print("Test mean :", X_test_scaled.mean())
print("Test std  :", X_test_scaled.std())

Train mean (≈0): -6.949481545266977e-14
Train std  (≈1): 1.000000000000053
Test mean : 0.04358887433900609
Test std  : 1.0162937249461212


In [1427]:
target_scalers = {}

# Ensure float arrays (avoid dtype surprises)
Y_train_scaled = Y_train.astype(float).copy()
Y_test_scaled  = Y_test.astype(float).copy()

for i, var in enumerate(TARGET_COLS):
    start = i * HORIZON
    end   = (i + 1) * HORIZON

    scaler = StandardScaler()

    # Fit ONLY on TRAIN targets for this variable
    scaler.fit(Y_train[:, start:end])

    # Transform train & test
    Y_train_scaled[:, start:end] = scaler.transform(
        Y_train[:, start:end]
    )
    Y_test_scaled[:, start:end] = scaler.transform(
        Y_test[:, start:end]
    )

    target_scalers[var] = scaler

In [1428]:
print("Test target mean:", Y_test_scaled.mean())
print("Test target std :", Y_test_scaled.std())

Test target mean: 0.08521854472096012
Test target std : 0.9497122390614124


In [1429]:
class WeatherLSTM(nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size=64,
        num_layers=2,
        horizon=7,
        num_targets=4,
        dropout=0.2
    ):
        super().__init__()

        self.horizon = horizon
        self.num_targets = num_targets
        self.output_size = horizon * num_targets

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )

        self.fc = nn.Linear(hidden_size, self.output_size)

    def forward(self, x):
        """
        x: (batch_size, LOOKBACK, input_size)
        """
        lstm_out, _ = self.lstm(x)          # (B, T, H)
        last_step = lstm_out[:, -1, :]      # (B, H)
        out = self.fc(last_step)            # (B, 28)
        return out

In [1430]:
device = "cuda" if torch.cuda.is_available() else "cpu"

num_targets = len(TARGET_COLS)

model = WeatherLSTM(
    input_size=len(FEATURE_COLS),
    hidden_size=64,
    num_layers=2,
    horizon=HORIZON,
    num_targets=num_targets,   # 🔥 FIX HERE
    dropout=0.2
).to(device)

In [1431]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-3
)

In [1432]:
BATCH_SIZE = 64

train_dataset = TensorDataset(
    torch.tensor(X_train_scaled, dtype=torch.float32),
    torch.tensor(Y_train_scaled, dtype=torch.float32)
)

test_dataset = TensorDataset(
    torch.tensor(X_test_scaled, dtype=torch.float32),
    torch.tensor(Y_test_scaled, dtype=torch.float32)
)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)


In [1433]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0

    for X_batch, Y_batch in loader:
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)

        optimizer.zero_grad()

        preds = model(X_batch)
        loss = criterion(preds, Y_batch)

        loss.backward()

        # gradient clipping (important for LSTM)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        total_loss += loss.item() * X_batch.size(0)

    return total_loss / len(loader.dataset)

In [1434]:
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for X_batch, Y_batch in loader:
            X_batch = X_batch.to(device)
            Y_batch = Y_batch.to(device)

            preds = model(X_batch)
            loss = criterion(preds, Y_batch)

            total_loss += loss.item() * X_batch.size(0)

    return total_loss / len(loader.dataset)

In [1435]:
EPOCHS = 50
PATIENCE = 4        
MIN_DELTA = 1e-4   
best_val_loss = float("inf")
epochs_no_improve = 0
best_epoch = 0


for epoch in range(1, EPOCHS + 1):
    train_loss = train_one_epoch(
        model,
        train_loader,
        optimizer,
        criterion,
        device
    )

    val_loss = evaluate(
        model,
        test_loader,
        criterion,
        device
    )

    print(
        f"Epoch {epoch:02d} | "
        f"Train MSE: {train_loss:.4f} | "
        f"Val MSE: {val_loss:.4f}"
    )

    # ---- Early stopping logic ----
    if val_loss < best_val_loss - MIN_DELTA:
        best_val_loss = val_loss
        epochs_no_improve = 0
        best_epoch = epoch

        # Save best model
        torch.save(model.state_dict(), f"{path}/best_lstm_model.pt")
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= PATIENCE:
        print(
            f"\nEarly stopping triggered at epoch {epoch}. "
            f"Best epoch was {best_epoch} "
            f"(Val MSE = {best_val_loss:.4f})"
        )
        break

Epoch 01 | Train MSE: 0.7595 | Val MSE: 0.4559
Epoch 02 | Train MSE: 0.6061 | Val MSE: 0.4502
Epoch 03 | Train MSE: 0.5911 | Val MSE: 0.4497
Epoch 04 | Train MSE: 0.5803 | Val MSE: 0.4367
Epoch 05 | Train MSE: 0.5717 | Val MSE: 0.4313
Epoch 06 | Train MSE: 0.5655 | Val MSE: 0.4197
Epoch 07 | Train MSE: 0.5571 | Val MSE: 0.4146
Epoch 08 | Train MSE: 0.5495 | Val MSE: 0.4241
Epoch 09 | Train MSE: 0.5431 | Val MSE: 0.4221
Epoch 10 | Train MSE: 0.5353 | Val MSE: 0.4127
Epoch 11 | Train MSE: 0.5305 | Val MSE: 0.4263
Epoch 12 | Train MSE: 0.5191 | Val MSE: 0.4265
Epoch 13 | Train MSE: 0.5099 | Val MSE: 0.4288
Epoch 14 | Train MSE: 0.5005 | Val MSE: 0.4250

Early stopping triggered at epoch 14. Best epoch was 10 (Val MSE = 0.4127)


## Interpretation

In [1436]:
def inverse_scale_predictions(
    Y_scaled,
    target_scalers,
    target_cols,
    horizon
):
    Y_real = Y_scaled.copy()

    for i, var in enumerate(target_cols):
        start = i * horizon
        end   = (i + 1) * horizon

        Y_real[:, start:end] = target_scalers[var].inverse_transform(
            Y_scaled[:, start:end]
        )

    return Y_real

In [1437]:
model.eval()

with torch.no_grad():
    X_test_tensor = torch.tensor(
        X_test_scaled, dtype=torch.float32
    ).to(device)

    Y_pred_scaled = model(X_test_tensor).cpu().numpy()

In [1438]:
Y_pred_real = inverse_scale_predictions(
    Y_pred_scaled,
    target_scalers,
    TARGET_COLS,
    horizon=HORIZON
)

Y_test_real = inverse_scale_predictions(
    Y_test_scaled,
    target_scalers,
    TARGET_COLS,
    horizon=HORIZON
)

In [1439]:
n_targets = len(TARGET_COLS)

Y_pred_real = Y_pred_real.reshape(-1, HORIZON, n_targets)
Y_test_real = Y_test_real.reshape(-1, HORIZON, n_targets)

In [1440]:
dates = test_df.index.values

test_dates = []
for i in range(LOOKBACK, len(test_df) - HORIZON):
    test_dates.append(dates[i])

test_dates = np.array(test_dates)

In [1441]:
sample_idx = 0  # first test sample

rows = []

for v_idx, var in enumerate(TARGET_COLS):
    rows.append({
        "variable": var,
        "pred_t+1": Y_pred_real[sample_idx, 0, v_idx],
        "true_t+1": Y_test_real[sample_idx, 0, v_idx],
    })

pd.DataFrame(rows)

,variable,pred_t+1,true_t+1
0,mean_temperature,12.871360,12.055556
1,max_temperature,18.721621,20.000000
2,min_temperature,7.321412,4.000000
3,total_precipitation,4.831858,0.000000
4,mean_windSpeed,2.767519,4.629996
5,mean_dewPoint,8.118137,0.166667
6,mean_visibility,8.221915,8.046700


In [1442]:
from sklearn.metrics import mean_absolute_error

v_idx = TARGET_COLS.index("mean_temperature")

mae_temp = mean_absolute_error(
    Y_test_real[:, :, v_idx].ravel(),
    Y_pred_real[:, :, v_idx].ravel()
)

print("MAE mean temperature (°C):", mae_temp)

MAE mean temperature (°C): 1.4761154545149975


In [1443]:
for h in range(HORIZON):
    mae_h = mean_absolute_error(
        Y_test_real[:, h, v_idx],
        Y_pred_real[:, h, v_idx]
    )
    print(f"Horizon t+{h+1}: MAE = {mae_h:.2f}")


Horizon t+1: MAE = 1.24
Horizon t+2: MAE = 1.42
Horizon t+3: MAE = 1.44
Horizon t+4: MAE = 1.50
Horizon t+5: MAE = 1.55
Horizon t+6: MAE = 1.58
Horizon t+7: MAE = 1.61


In [1444]:
# Persistence: predict last observed value for all horizons
Y_persist = np.repeat(
    Y_test_real[:, 0:1, v_idx],  # last known temp
    HORIZON,
    axis=1
)

mae_persist = mean_absolute_error(
    Y_test_real[:, :, v_idx].ravel(),
    Y_persist.ravel()
)

print("Persistence MAE:", mae_persist)

Persistence MAE: 1.6603543743078624


In [1445]:
import joblib

joblib.dump({
    "scaler": feature_scaler,
    "feature_cols": FEATURE_COLS
}, f"{path}/feature_scaler_bundle.pkl")
joblib.dump(target_scalers, f"{path}/target_scalers.pkl")

['artifacts/sale/target_scalers.pkl']